# Well-being and Nutrition

In this notebook, we would like to study the link between well-being and nutrition.

More precisely, our goal is to analyze if it is possible to predict the food consumption of an area from publicly available well-being data. We will study results at the level of London city wards, as this is the highest granularity we could find for general well-being data.

## Context

From [Tesco Grocery 1.0, a large-scale dataset of grocery purchases in London](https://www.nature.com/articles/s41597-020-0397-7) paper, we were able to understand how the nutritional data of the average product of an area could help to predict the prevalence of common diseases such as diabetes. We were interested in the relationship between nutrition and general well-being indicators. Data on general well-being is publically available for any big city. On the other hand, nutritional purchases data has to be anonymized and is distributed among different private companies such as Tesco. We therefore aim to predict the nutritional information of the average product per area based on well-being measures.

The City of London has conducted many studies and surveys on the well-being of its inhabitants, so it was not too difficult to find the required datasets to follow our studies. However, it would have been preferable to work with datasets at a higher granularity (LSOA or MSOA), as some phenomena might be smoothened out by bigger areas.

## Data

Our study will use two datasets:

1) The data provided by the Tesco Grocery 1.0 paper (`year_osward_grocery`) giving the nutritional informations of the average product per ward, the representativeness of Tesco's data and some additional information such as population, gender, and age.

2) The well-being probability score dataset from [London datastore](https://data.london.gov.uk/). This data is from 2013. However, we can make the assumption that the well-being values do not significantly change in two years. In this file, they compute the total well-being per area in 2013 based on different categories (Health, Education, financial aspects, etc.). The spreadsheet is interactive, and therefore allows to weigh each category differently to compute the final well-being score. We decided to put the same weight to every category except the subjective well-being (self-stated happiness feeling). We set it to zero, as we want to make predictions based on objective and measurable input. The well-being Index Score (which we will rename to Total Well-being Score) is thus a mean of all the objective variables.

You can find the well-being dataframe in the `data` folder (`./data/london-ward-well-being-probability-scores.xls`). We used the "score" page which gives the scores of the different variables for each area.

The scores for each feature and each area are calculated as follows: $score = \frac{data(area) - data(England \, and \, Wales)}{Standard \, Deviation}$

Therefore, if the score is positive, it means that the well-being variable is higher than UK's mean value for this area.

We provide a brief description of the fields you will encounter:

- `area_id`: the number for each area of London at ward scale.

- `Life_Expectancy`: Index scores were reversed so higher life expectancy equals better well-being. Source: ONS mortality data and GLA population projections, GLA Calculations.

- `Childhood Obesity`: Children with a BMI greater than or equal to the 95th centile of the British 1990 growth reference (UK90) BMI distribution have been classified as obese. Source: National Obesity Observatory.		

- `Incapacity Benefit rate`: Incapacity Benefit (IB) is paid to people who are incapable of work and who meet certain contribution conditions. Severe Disablement Allowance (SDA) is paid to those unable to work for 28 weeks in a row or more because of illness or disability. Source: IB/SDA from DWP, Population from GLA projections.

- `Unemployement rate`: Percentage of working-age residents claiming Jobseeker's Allowance (JSA) or National Insurance Credits. JSA is a benefit payable to unemployed people. In general, to be entitled to claim a person must be available for work, be actively seeking work, and have entered into a Jobseeker's Agreement with Jobcentre Plus. Source: JSA from DWP, Population from GLA projections.

- `Crime rate - index`: Index scores of overall notifiable offences per 1,000 daytime population. Source: MPS, Home Office, and ONS Workday population 2011 Census.

- `Deliberate fires`: Rate of all Deliberate Fire incidents (arson) recorded by the London Fire & Emergency Planning Authority per 1,000 population. Source: LFEPA via LASS team at GLA, and Population from GLA 2012 projections.

- `Average Capped GCSE and Equivalent Point Score Per Pupil`: GCSE and Equivalent point scores for pupils at the end of Key Stage 4 (KS4) in maintained schools (Referenced by Location of Pupil Residence). Ward data calculated by apportioning Lower Layer Super Output Area (LSOA) data. Index scores were reversed so higher GCSE scores equals better well-being. Source: DfE (on Neighbourhood Statistics).

- `Unauthorised Absence in All Schools (%)`: Pupil Absence in all maintained schools (Referenced by Location of Pupil Residence). Unauthorised Absence is absence without permission from a teacher or other authorised representative of the school. Ward data calculated by apportioning Lower Layer Super Output Area (LSOA) data. Source: DfE (on Neighbourhood Statistics).

- `Dependent children in out-of-work families`: Data represent the number of children dependent on a parent or guardian who is claiming one or a combination of out-of-work benefits. Source: DWP and GLA 2012 population projections.

- `Public Transport Accessibility`: take into account walk access time and service availability. The method is essentially a way of measuring the density of the public transport network at any location within Greater London. Each area was given an average score out of 8, where 8 is the highest level of accessibility. Open space was removed from the data as no population lives there. (Source: PTAL contours from Transport for London, further calculations by [GLA](http://www.tfl.gov.uk/businessandpartners/syndication/16493.aspx))

- `Homes with access to open space & nature, and % greenspace`:  There are four types of public open space according to the 2011 London Plan. Homes further away than the maximum recommended distance are considered to be deficient in access to that type of public open space. Access to nature measures the proportion of homes with good access to nature. The final measure is the proportion of area that is greenspace within the ward. In these combined scores, each of the three measures has been given a weight of 33%. (Source: Greenspace Information for Greater London, and Ordnance Survey)

## Import modules

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import pprint

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import cross_validate, train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

import statsmodels.formula.api as smf
import statsmodels.api as sm

from statsmodels.stats import diagnostic
from statsmodels.stats import diagnostic
from statsmodels.stats.weightstats import ttost_ind
from statsmodels.stats.multitest import multipletests

from utils import calculate_nutripoints

pp = pprint.PrettyPrinter(depth=4)
np.random.seed(42)

sns.set_theme('notebook')
sns.color_palette("colorblind")
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

## I) Import data

In [ ]:
year_grocery = pd.read_csv("data/year_osward_grocery.csv")
display(year_grocery.head())

# for wellbeing_score, we have to get the scores of the individual variable in the "Scores" sheet
# and the final score, which is the mean of all the variables, in the "Ranked" sheet

wellbeing_scores = pd.read_excel(
    "data/london-ward-well-being-probability-scores.xls", sheet_name="Scores", header=[0, 1])
display(wellbeing_scores.head())

wellbeing_total_scores = pd.read_excel(
    "data/london-ward-well-being-probability-scores.xls", sheet_name="Ranked", header=[3], usecols="B:C")
display(wellbeing_total_scores.head())

## II) Preprocessing

### A) Filter data

We consider only 80% of the areas, the ones with the highest representativeness. This will probably allow us to have more interesting and robust results.

In [ ]:
PERCENTAGE_SPLIT_REPRESENTATIVENESS = 0.8
N = len(year_grocery)

year_grocery = year_grocery.nlargest(
    int(PERCENTAGE_SPLIT_REPRESENTATIVENESS * N), 'representativeness_norm')

### B) Compute Nutripoints

To predict the nutritional variables of the average product per area, we thought it would be easier to assess a single variable representative of the individual ones per nutrients. We called it **Nutripoints**. 

We based our computation of the Nutripoints on the definition of the French national Nutri-Score. The bigger it is, the worse is the average product regarding the level of sugar, saturate, sodium and total energy it has. On the other hand, if it is negative (which is almost impossible), it represents products with higher level of proteins and fibers than "bad nutrients" (salt, sugar, saturate). We adapted the different ranges of the official formula to our database as we have average products and therefore a lower variance (small interval). You can see the detailed function in the file: `utils.py`

We then add a column with the computed Nutripoints to our `year_grocery` dataset, which is the one with the nutritional values, assessing the nutritional quality of the average product of each London ward.

In [ ]:
min_grocery = year_grocery[['energy_tot', 'saturate', 'salt',
                            'sugar', 'f_fruit_veg', 'fibre', 'protein']].min(axis=0)

max_grocery = year_grocery[['energy_tot', 'saturate', 'salt',
                            'sugar', 'f_fruit_veg', 'fibre', 'protein']].max(axis=0)

year_grocery["nutripoints"] = year_grocery.apply(
    lambda row: calculate_nutripoints(row, min_grocery, max_grocery), axis=1)

year_grocery["nutripoints"].describe()

In [ ]:
display(year_grocery.head())

In [ ]:
print(year_grocery['area_id'].duplicated().any())

Our Nutripoints are distributed from -1.5 to 15; having an interval of 16.5. However, as the mean is at 7.7 and the 1st and 3rd quartiles are at respectively 6 and 9.5, we see that most of the values are aggregated between these two values. The two outer quartiles have spread datapoints. The min and max values must be outliers.

This will actually complicate our study, as there might not be many differences or at least considerable differences between the areas. It is probably because we have the average product per area.

[//]: # "Indeed, London's inhabitants buy mainly the same type of foods but not the same quality, according to their budget."

###  C) Filter wellbeing data

We filter the columns in order to keep only the information from the last year of the dataset, 2013. 

In [ ]:
# only keep the columns containing 2013 and the code and name of the ward

wellbeing_scores = wellbeing_scores.loc[:, (slice(
    None), [2013, "New ward code", "Ward name"])].dropna(how="all")

# We first had two levels of indexes, one with the years and the other with the type of variable
# Now that we only keep 2013, we drop the second level of line indexes.
wellbeing_scores = wellbeing_scores.droplevel(1, axis=1)

display(wellbeing_scores.head())

In [ ]:
wellbeing_scores_columns = wellbeing_scores.columns.values.tolist()
print(wellbeing_scores_columns)

### D) Merge datasets

After this step, we will be able to do some exploratory data analysis, understanding the link between well-being and nutrition. To do so, we first merge the two datasets keeping only the columns that are interesting to put in parallel.

First of all, we merge the two well-being datasets in order to have the Total Well-being score (column `Index Score 2013`) with all its components used to compute it. 

Then, we select the energy for each nutrient, the entropy and the Nutripoints from `year_grocery`. These variables will be used to represent the nutritional information of the Nutrition dataset. 

It is important to check for null values to know if our dataset is complete.

In [ ]:
# Merging the two wellbeing datasets according to the ward name in order to have the total score in only one wellbeing dataset
wellbeing_scores = pd.merge(
    left=wellbeing_scores, right=wellbeing_total_scores, left_on='Ward name', right_on="Ward")

wellbeing_scores = wellbeing_scores.drop("Ward", axis=1)

display(wellbeing_scores.head())

In [ ]:
RENAMED_COLUMNS = {
    'Incapacity Benefit rate': 'Incapacity Benefit Rate',
    'Unemployment rate': 'Unemployment Rate',
    'Average Capped GCSE and Equivalent Point Score Per Pupil': 'Average GCSE',
    'Unauthorised Absence in All Schools (%)': 'Unauthorised Absence',
    'Dependent children in out-of-work families': 'Dependent Children',
    'Public Transport Accessibility': 'Public Transport Access',
    'Homes with access to open space & nature, and % greenspace': 'Access to Nature',
    'Subjective well-being average score': 'Happiness Score',
    'Index Score 2013': 'Wellbeing Score',
    'Crime rate - Index': 'Crime Rate'
}

wellbeing_scores = wellbeing_scores.rename(
    columns=RENAMED_COLUMNS, errors="raise")

In [ ]:
# Selection of the columns of interest in year_grocery dataset
list_column = ["area_id", "energy_tot", "energy_fat", "energy_saturate", "energy_sugar", "energy_protein",
               "energy_carb", "energy_fibre", "energy_alcohol", "h_nutrients_calories", "nutripoints"]

year_grocery = year_grocery.loc[:,
                                year_grocery.columns.isin(list(list_column))]
display(year_grocery.head())

In [ ]:
# Merging the 2 datasets of interest: year grocery and wellbeing scores according to the ward name
wellbeing_grocery = pd.merge(
    left=year_grocery, right=wellbeing_scores, left_on='area_id', right_on="New ward code")

wellbeing_grocery = wellbeing_grocery.drop("New ward code", axis=1)
display(wellbeing_grocery.head())

In [ ]:
# Checking for any null values in the new dataset
wellbeing_grocery_columns = wellbeing_grocery.columns.values.tolist()
wellbeing_grocery.isnull().any()

Our dataset of study is complete as there is no null values in any of our columns.

In [ ]:
# Understanding better how each variable is organized: its min, max, mean, standard deviation and quartiles
wellbeing_grocery.describe()

For our analysis, we will create a database from our merged one (`wellbeing_grocery`) keeping only the numerical variables. We also standardise the values in order to be able to compare them. 

In [ ]:
# List of columns of interest in the wellbeing dataset
COLUMNS_SCORES = [
    'Life Expectancy',
    'Childhood Obesity',
    'Incapacity Benefit Rate',
    'Unemployment Rate',
    'Crime Rate',
    'Deliberate Fires',
    'Average GCSE',
    'Unauthorised Absence',
    'Dependent Children',
    'Public Transport Access',
    'Access to Nature',
    'Happiness Score',
    'Wellbeing Score'
]

# List of columns of interest in the nutritional dataset
COLUMNS_GROCERY = [
    'energy_fat',
    'energy_saturate',
    'energy_sugar',
    'energy_protein',
    'energy_carb',
    'energy_fibre',
    'energy_alcohol',
    'energy_tot',
    'h_nutrients_calories',
    'nutripoints'
]

COLUMNS = COLUMNS_GROCERY + COLUMNS_SCORES

# Selection of the numerical columns of interest in the wellbeing_grocery dataset
wellbeing_grocery_analysis = wellbeing_grocery[COLUMNS].copy()

In [ ]:
# Standardizing features by removing the mean and scaling to unit variance
scaler = StandardScaler()
wellbeing_grocery_analysis[wellbeing_grocery_analysis.columns] = scaler.fit_transform(wellbeing_grocery_analysis
                                                                                      [wellbeing_grocery_analysis.columns])
wellbeing_grocery_analysis.describe()

## III) Exploratory Data Analysis

In this part, we will do some tests putting in parallel the well-being variables and the nutritional ones to analyse if they are correlated and therefore if we could really use one to predict the other. 

### A) Repartition of the values for each variable

In [ ]:
# Explanatory data analysis: visually show the distribution of the values for each variable thanks to boxplot
fig, ax = plt.subplots(4, 6, figsize=(16, 8), sharey=False)

for i in range(len(COLUMNS)):
    sbplt = ax[int(i/6), i % 6]

    sns.boxplot(data=wellbeing_grocery_analysis.iloc[:, i], ax=sbplt)
    sbplt.set_xlabel('')
    sbplt.set_ylabel('')
    sbplt.set_title(COLUMNS[i], loc='center', wrap=True)

fig.tight_layout()
fig.subplots_adjust(top=0.9)

fig.suptitle('Boxplot for each wellbeing and nutritional variable', fontsize=18)

These box plots give us a general view of the distribution of each variable. Since we would like to cluster nutrient data for specific areas and predict nutritional habits according to well-being characteristics, it is important to observe differences between areas for the various features. Here, the nutritional as well as the well-being features seem to take many values depending on the ward. Therefore, it is meaningful to understand if these differences in the nutritional variables are linked to differences in the well-being features. Moreover, some outliers can sometimes be seen, for example for the energy_saturate and the crime rate. 

In [ ]:
# Exploratory data analysis: distribution of the different variables by counting the number of observations per value
fig, ax = plt.subplots(4, 6, figsize=(16, 8), sharey=False)

for i in range(len(COLUMNS)):
    sbplt = ax[int(i / 6), i % 6]

    sns.histplot(wellbeing_grocery_analysis.iloc[:, i], ax=sbplt)
    sbplt.set_xlabel('')
    sbplt.set_ylabel('')
    sbplt.set_title(COLUMNS[i], wrap=True)

fig.tight_layout()
fig.subplots_adjust(top=0.9)

fig.suptitle('Distributions of separate features', fontsize=18);

It seems that some variables, as "Access to nature" and "Life Expectancy", follow a normal distribution. Let's check our assumption!

In [ ]:
# Explanatory data analysis: distribution of the different variables by computing the proportion of values below or equal
# to a certain value
fig, ax = plt.subplots(4, 6, figsize=(16, 8), sharey=False)

for i in range(len(COLUMNS)):
    sbplt = ax[int(i / 6), i % 6]

    sns.ecdfplot(wellbeing_grocery_analysis.iloc[:, i], ax=sbplt)
    sbplt.set_xlabel('')
    sbplt.set_ylabel('')
    sbplt.set_title(COLUMNS[i], wrap=True)

fig.tight_layout()
fig.subplots_adjust(top=0.9)

fig.suptitle('CDF of separate features', fontsize=18);

We observe districumulative distribution function of the format of a normal distribution, it seems that certain variables follow this kind of distribution.

We will now test it more thoroughly using three different tests to check normality. We set the significance level to $\alpha = 0.05$, and only accept the columns that pass every test.

In [ ]:
# check for any normal distribution
from collections import defaultdict
from scipy.stats import shapiro, normaltest
alpha = 0.05


results = defaultdict(list)
# normality test
for col in COLUMNS:
    data = wellbeing_grocery_analysis[col]
    stat_shap, pvalue_shap = shapiro(data)
    stat_normal, pvalue_normal = normaltest(data)
    stat_kstest, pvalue_kstest = diagnostic.kstest_normal(data, dist='norm')
    passed_tests = np.sum(
        np.array([pvalue_shap, pvalue_normal, pvalue_kstest]) > alpha)
    if passed_tests:
        # failed to reject H0
        results['Factor'].append(col)
        results['Passed Tests'].append(passed_tests)

display(pd.DataFrame(results).sort_values("Passed Tests", ascending=False))

The second value returned by the function is the p-value. When the p_value < 0.05 -> we can reject the null hypothesis that the data comes from a normal distribution!
Here, we observe that:

- energy_carb
- energy_tot
- Life Expectancy
- Access to Nature
- Well-being Score

have a p-value > 0.05 for all normality test.

Therefore, the null hypothesis is not rejected and we can say that those variables follow a normal distribution.

### B) Correlation study

After checking how each variable is distributed: its interval, mean and standard deviation, we will now observe the correlation between our Nutripoints, representative of nutritional informations, and each well-being variable.

In [ ]:
# Computing and plotting the correlation between the well-being scores and nutripoints
correlation = wellbeing_grocery_analysis[COLUMNS_SCORES +
                                         ['nutripoints']].corr()
correlation.to_pickle("plot_data/wellbeing_correlation.pkl")

fig = plt.figure(figsize=(10, 5))

sns.heatmap(correlation);

This heatmap helps us visualize the correlation between the different variables. If we focus on the correlation between Nutripoints and wellbeing features (last row), we can see actually that the values are quite low because the color is mainly purple corresponding to the negative correlation values and they are all below 0.4 which is not high. This is not a good augur for the continuation of the project.

In [ ]:
# Computing the correlation between the different variables from the dataset according to the spearman method

correlation = wellbeing_grocery_analysis.corr(method="spearman")

From the results obtained thanks to the heatmap, we decided to analyze if it was only the nutripoints that did not have a high correlation with the well-being variables or if we could also observe this phenomenom for the other nutritional variables (energy and entropy).

In [ ]:
# Plotting the correlation between the nutritional and the well-being variables

fig, ax = plt.subplots(5, 2, figsize=(15, 10), sharex=True, sharey=True)

for i in range(len(COLUMNS_GROCERY)):
    sbplt = ax[int(i / 2), i % 2]

    correlation[COLUMNS_GROCERY[i]][COLUMNS_SCORES].plot.bar(
        x=None, y=None, width=0.8, legend=None, ax=sbplt)
    sbplt.set_title(wellbeing_grocery_analysis.columns[i], wrap=True)


fig.text(0.08, 0.5, 'Spearman R', va='center', rotation='vertical')
fig.subplots_adjust(top=0.9)

fig.suptitle("Spearman Correlation", fontsize=18);

We can observe on the graphs above that the correlation is low for all the nutritional variables. They never exceed 0.5 except for energy_fibre. Therefore, it seems that it is going to be difficult to predict Nutripoints from the well-being variables because they only have a few impact on the nutritional aspect of an area. Especially the Happiness Score is hardly correlated to any nutritional value.

### C) Regression analysis

We have seen that well-being variables are not very correlated to the nutritional ones. However, to be sure about it, we want to test some regressions and observe the R-squared and p-values for each coefficient.

We will try predicting entropy and fibre (the nutritional values with the highest correlation), with the well-being variables that seem, in our opinion, the most linked to nutrition.

Indeed, we could think that if unemployement is high, people have less money to spend on healthy food. Furthermore, childhood obesity must be linked to unhealthy food, high in fat and sugar.

In [ ]:
COLUMNS_CORRELATED = ["Life Expectancy", "Incapacity Benefit Rate",
                      "Unemployment Rate", "Crime Rate", "Childhood Obesity", "Access to Nature"]

# Selection of the wellbeing features that seem most correlated to nutrition
reg_features = 'Q("Life Expectancy") + Q("Incapacity Benefit Rate") + Q("Unemployment Rate") + Q("Crime Rate") \
+ Q("Childhood Obesity") + Q("Access to Nature")'

# Linear regression to predict entropy from those features
mod = smf.ols(formula='h_nutrients_calories ~ ' +
              reg_features, data=wellbeing_grocery_analysis)
res = mod.fit()
print(res.summary());

In [ ]:
# Linear regression to predict energy_fibre
mod = smf.ols(formula='energy_fibre ~ ' + reg_features,
              data=wellbeing_grocery_analysis)
res = mod.fit()
print(res.summary())

R-squared are quite low meaning that either 20% or 40% of the data can be predicted. Furthermore, the p-values of the the coefficients should be below 0.05 in order to be meaningful, which they are not, unless for childhood obesity when predicting the entropy.

### D) PCA

Now, we do a PCA to project the well-being data into a 2D plane.

In [ ]:
# PCA on the nutrition features

wellbeing_grocery_analysis = wellbeing_grocery[COLUMNS_GROCERY]\
    .dropna().copy()

wellbeing_grocery_reduced_pca = PCA(n_components=2).fit_transform(
    wellbeing_grocery_analysis)

In [ ]:
# Labels computed as explained above
labels = wellbeing_grocery.apply(
    lambda row: "g" if row['Wellbeing Score'] >= 0 else "r", axis=1)

# Plot the data reduced to a 2D plane with PCA
plt.figure(figsize=(14, 5))
plt.scatter(wellbeing_grocery_reduced_pca[:, 0],
            wellbeing_grocery_reduced_pca[:, 1], c=labels, alpha=0.6)
plt.xlabel('component 1')
plt.ylabel('component 2')

plt.title("PCA projection of nutritional values", fontsize=18);

In [ ]:
# Election of the column we would like to determine the number of clusters for: here it is the Nutripoints
year_grocery = year_grocery.sort_values(by=["nutripoints"])

columns_kmeans = ['nutripoints']
year_grocery_kmeans = year_grocery[columns_kmeans].copy()

# Function to plot the sse


def plot_sse(X, start=2, end=11):
    sse = []
    for k in range(start, end):
        # Assign the labels to the clusters
        kmeans = KMeans(n_clusters=k, random_state=10).fit(X)
        sse.append({"k": k, "sse": kmeans.inertia_})
    sse = pd.DataFrame(sse)
    # Plot the data
    plt.plot(sse.k, sse.sse)
    plt.xlabel("K")
    plt.ylabel("Sum of Squared Errors")
    plt.title("Sum of Squared Errors curve to find the optimal k", fontsize=13)


plot_sse(year_grocery_kmeans)

### E) k-Means Clustering to Label Data

In order to better understand how nutripoints are spread and to see if we can identify some clusters linked to well-being features, we will process the data using K-means.

We know that a positive value corresponds to a better score than the England average whereas the negative are worse. We could then label our datapoints with respect to their wellbeing score (value comptued from all well-being variables, higher corresponds to a better well-being estimate). We set the color of positive well-being scores to green, negative to red. Unfortunately, the nutritional values do not seem create clusters or visually separate depending on the well-being score.

We will now label the data based on its nutripoints. We apply this to `year_grocery` as we want to take the distribution for all the wards and not only the ones for which we also have wellbeing values.

To understand how many clusters we can determine for the nutripoints, we first plot the sum of squared erros curve for each number of cluster in order to find the optimal number k. It is called the elbow method. Indeed, the point where the curve actually breaks should be the best number of clusters for this dataset. 
To confirm our choice, we will also do the silhouette score. This curve represents the score assessed for each number of clusters considering the separation distance between the resulting clusters. A peak on the curve indicates that the score reached its global maximum at the optimal k. 

In [ ]:
# 2nd method to choose k: the silhouette score
silhouettes = []

# Trying for k between 2 and 10
for k in range(2, 11):
    # Cluster the data and assign the labels
    labels = KMeans(n_clusters=k, random_state=10).fit_predict(
        year_grocery_kmeans)
    # Get the Silhouette score
    score = silhouette_score(year_grocery_kmeans, labels)
    silhouettes.append({"k": k, "score": score})

# Convert to dataframe
silhouettes = pd.DataFrame(silhouettes)

# Plot the data
plt.plot(silhouettes.k, silhouettes.score)
plt.xlabel("K")
plt.ylabel("Silhouette score")
plt.title("Silhouette score curve to find the optimal k", fontsize=13);

On the sse plot, we can observe an elbow for k=3 and k=5. On the other hand, when plotting the silhouette score, we see that the curve goes down for k=3 but the silhouette score is higher for k=5. Therefore, we decided to divide the nutripoints into 5 clusters. To do so, we use the K-means method. We also rank the areas from the lowest nutripoints to the biggest.

In [ ]:
# Clustering the data with the current number of clusters
kmean = KMeans(n_clusters=5, random_state=42).fit(year_grocery_kmeans)

# sort labels according to center
idx = np.argsort(kmean.cluster_centers_.sum(axis=1))
lut = np.zeros_like(idx)
lut[idx] = np.arange(5)

# Plotting the data by using the labels as color
year_grocery['nutri_class'] = lut[kmean.labels_]

In [ ]:
display(year_grocery.head())
print(year_grocery.shape)
print(wellbeing_grocery.shape)

In [ ]:
year_grocery.to_pickle("data/grocery_nutripoints.pkl")

The `year_grocery` database with the Nutripoints and the class they are assigned to will be saved into pickle in order to be reused in another file such as  `Nutripoints` where we analyse in more details the distribution of the different clusters and the notebook `fastfood` where we want to study another correlation.

In [ ]:
# Merging the 2 datasets of interest: year grocery and wellbeing scores according to the ward name
wellbeing_grocery_pca = pd.merge(
    left=year_grocery, right=wellbeing_scores, left_on='area_id', right_on="New ward code")

wellbeing_grocery_pca = wellbeing_grocery_pca.drop("New ward code", axis=1)

In [ ]:
# PCA on the wellbeing features

wellbeing_grocery_analysis = wellbeing_grocery_pca[COLUMNS_CORRELATED]\
    .dropna().copy()

wellbeing_grocery_reduced_pca = PCA(n_components=2).fit_transform(
    wellbeing_grocery_analysis)

In [ ]:
# Plot the data reduced to a 2D plane with PCA
color = wellbeing_grocery_pca["nutri_class"].values
plt.figure(figsize=(14, 6))
# plt.scatter(wellbeing_grocery_reduced_pca[:, 0],
#            wellbeing_grocery_reduced_pca[:, 1], c=color, alpha=0.6)

plt.scatter(wellbeing_grocery_reduced_pca[:, 0],
            wellbeing_grocery_reduced_pca[:, 1],
            c=color, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('Paired', 5))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar()

plt.title("PCA projection of wellbeing values", fontsize=18);

The different colors correspond to the nutri_class every datapoint is in. Again, the correlated features do not seem to be enough to separate the nutripoints into clear clusters.

### F) Equivalence Test

Our visual analysis suggests that there is no significant relationship between a general well-being score and nutritional quality of an area. To formalize this intuition, we will perform an equivalence test. Equivalence tests are a variation of hypothesis tests used to draw statistical inferences from observed data. In equivalence tests, the null hypothesis is defined as an effect large enough to be deemed interesting, specified by an equivalence bound. The alternative hypothesis is any effect that is less extreme than said equivalence bound. The observed data is statistically compared against the equivalence bounds.

If the statistical test indicates the observed data is surprising, assuming that true effects at least as extreme as the equivalence bounds, a Neyman-Pearson approach to statistical inferences can be used to reject effect sizes larger than the equivalence bounds with a pre-specified Type 1 error rate.

In [ ]:
def classify_wellbeing(row, column):
    if row[column] > 0:
        return "prosperous"
    else:
        return "unpropitious"


wellbeing_grocery["prosperity"] = wellbeing_grocery.apply(
    lambda row: classify_wellbeing(row, "Wellbeing Score"), axis=1)

In [ ]:
wellbeing_grocery[['nutripoints', 'prosperity']
                  ].boxplot(by='prosperity', figsize=(10, 5))

plt.show()

Assuming that the threshold value for a practical difference in nutripoint is 1, the null and alternative hypotheses would be:

$H_0: |\mu_1 - \mu_2| \geq 1$

$H_1: |\mu_1 - \mu_2| < 1$

In [ ]:
p_values = []

for col in COLUMNS_SCORES:
    median_col = wellbeing_grocery[col].median()

    if col in ("Average GCSE",
               "Public Transport Access",
               "Access to Nature",
               "Happiness Score",
               "Wellbeing Score"):
        wellbeing_grocery["prosperity"] = wellbeing_grocery.apply(
            lambda row: classify_wellbeing(row, col), axis=1)
    else:
        wellbeing_grocery["prosperity"] = wellbeing_grocery.apply(
            lambda row: classify_wellbeing(-1 * row, col), axis=1)

    treated = wellbeing_grocery[wellbeing_grocery['prosperity']
                                == "prosperous"]['nutripoints']
    control = wellbeing_grocery[wellbeing_grocery['prosperity']
                                == "unpropitious"]['nutripoints']

    p_value_non_equivalence, lower_threshold_test, upp_threshold_test = ttost_ind(
        x1=control, x2=treated, low=-1, upp=1)

    p_values.append(p_value_non_equivalence)


# multiple tests correction
rejected, p_values_corrected, _, _ = multipletests(p_values)

for p_value_non_equivalence, col in zip(p_values_corrected, COLUMNS_SCORES):

    test_result = "H_0 rejected" if p_value_non_equivalence < 0.05 else "H_0 not rejected"
    print(f"{test_result} for {col}, p-value: {p_value_non_equivalence:.2e}")
    # print(lower_threshold_test)
    # print(upp_threshold_test)

As in traditional hypothesis testing, the aim of our equivalence test is to reject the null hypothesis in order to conclude that the alternative hypothesis is demonstrated beyond reasonable doubt. In our case, if we can reject the null hypothesis $H_0$ (that the difference between the two groups is higher than or equal to 1), we are allowed to conclude that the mean difference between the groups at the population level is below 1, a value defined as trivial.
The `treatment` is having a positive well-being score.

However, we can not conclude that the effect of the experimental treatment is practically insignificant, as the null hypothesis is not rejected for any feature.

## IV) Predictive Models

We have seen that there are some columns with a considerable correlation between well-being features and our computed nutripoints. Therefore, we will now try to predict the nutripoints from these well-being features and assess the quality of the model.

We will test 3 different models to see if we can can successfully predict the nutripoins from the well-being features.

In [ ]:
X = wellbeing_grocery[COLUMNS_CORRELATED]
y = wellbeing_grocery[["nutripoints"]]

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)
X_tr = X_tr.copy()
X_te = X_te.copy()

X_tr[X_tr.columns] = scaler_X.fit_transform(X_tr[X_tr.columns])
X_te[X_te.columns] = scaler_X.transform(X_te[X_te.columns])

y_tr = scaler_y.fit_transform(y_tr).ravel()
y_te = scaler_y.transform(y_te).ravel()

print(X_tr.shape, X_te.shape)

We  will train a gradient boosting regressor. We decided to start with this model given our low amount of data samples. Ensemble methods like the gradient boosting regressor are known to be much more robust against over-fitting than simpler models like linear regression.

### Train Gradient Boosting Regressor

In [ ]:
# Regression models
models = {}

score_mse = 'neg_mean_squared_error'

Life Expectancy, Unemployment Rate, Deliberate Fires, Public Transport Access and Happiness Score are the selected variables for prediction of Nutripoints. Some features meaningful for the Linear Regression such as Well-being Score and GCSE are not here and on the other hand Happiness Score was not significant in the previous regression. 

In [ ]:
## Gradient Boosting Regression ##

n_estim_list = np.arange(1, 101)

mse_train_gradboost = []
mse_val_gradboost = []

for n_estim in n_estim_list:
    gradboost = GradientBoostingRegressor(n_estimators=n_estim)

    scores_gradboost = cross_validate(gradboost, X_tr, y_tr, cv=10, return_train_score=True,
                                      scoring=(score_mse))

    mse_train_gradboost.append(-np.mean(scores_gradboost['train_score']))
    mse_val_gradboost.append(-np.mean(scores_gradboost['test_score']))

In [ ]:
ind = np.argmin(mse_val_gradboost)
opt_param_gradboost = n_estim_list[ind]
opt_mse_val_gradboost = mse_val_gradboost[ind]

print("n =", opt_param_gradboost)

models["gradboost"] = (opt_mse_val_gradboost, opt_param_gradboost)

In [ ]:
## Ridge Regression ##

alphas_list = np.linspace(start=0.1, stop=10, num=100)

mse_train_ridge = []
mse_val_ridge = []

for a in alphas_list:
    ridge = Ridge(alpha=a)

    scores_ridge = cross_validate(ridge, X_tr, y_tr, cv=10, return_train_score=True,
                                  scoring=(score_mse))

    mse_train_ridge.append(-np.mean(scores_ridge['train_score']))
    mse_val_ridge.append(-np.mean(scores_ridge['test_score']))

In [ ]:
ind = np.argmin(mse_val_ridge)
opt_param_ridge = alphas_list[ind]
opt_mse_val_ridge = mse_val_ridge[ind]

print("alpha =", opt_param_ridge)

models["ridge"] = (opt_mse_val_ridge, opt_param_ridge)

In [ ]:
## Random Forest Regression ##

mse_train_rf = []
mse_val_rf = []

for n_estim in n_estim_list:
    rf = RandomForestRegressor(n_estimators=n_estim)

    scores_rf = cross_validate(rf, X_tr, y_tr, cv=10, return_train_score=True,
                               scoring=(score_mse))

    mse_train_rf.append(-np.mean(scores_rf['train_score']))
    mse_val_rf.append(-np.mean(scores_rf['test_score']))

In [ ]:
ind = np.argmin(mse_val_rf)
opt_param_rf = n_estim_list[ind]
opt_mse_val_rf = mse_val_rf[ind]

print("n_estimators =", opt_param_rf)
models["rf"] = (opt_mse_val_rf, opt_param_rf)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 4), sharey=True)

# GradBoost

line_1 = ax[0].plot(n_estim_list, mse_train_gradboost, label='training error')
line_2 = ax[0].plot(n_estim_list, mse_val_gradboost, label='validation error')
ax[0].set_xlabel('Number of Estimators')
ax[0].set_ylabel('MSE')
ax[0].set_title('GradBoost Regressor')
ax[0].legend()

##############################################

# Ridge

line_1 = ax[1].plot(alphas_list, mse_train_ridge, label='training error')
line_2 = ax[1].plot(alphas_list, mse_val_ridge, label='validation error')
ax[1].set_xlabel('Alpha')
ax[1].set_ylabel('MSE')
ax[1].legend()
ax[1].set_title('Ridge Regressor')


##############################################

# RandomForest

# print(x_arr, mse_train)
line_1 = ax[2].plot(n_estim_list, mse_train_rf, label='training error')
line_2 = ax[2].plot(n_estim_list, mse_val_rf, label='validation error')
ax[2].set_xlabel('Number of Estimators')
ax[2].set_ylabel('MSE')
ax[2].set_title('Random Forest Regressor')
ax[2].legend()

In [ ]:
# Model selection

print(models)

In [ ]:
# Plot the results for the selected model

gradboost = RandomForestRegressor(n_estimators=opt_param_rf)
gradboost.fit(X_tr, y_tr)

y_pred = gradboost.predict(X_te)

fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(y_te, y_pred, edgecolors=(0, 0, 0))
p1 = max(max(y_pred), max(y_te))
p2 = min(min(y_pred), min(y_te))
plt.plot([p1, p2], [p1, p2], 'b-', c='r')
ax.set_xlabel('Original')
ax.set_ylabel('Predicted')
plt.show()

We finally test our results on the testing set and plot the results. The dots should be aligned close to the red line if the predictions were accurate.

We see that the best model's fit does not achieve a very good accuracy. Although the bias does not seem very high, the model does not achieve to grasp the variance of the data.

## V) Conclusions

We conclude that we were not able to successfully fit a generalizable model to predict the nutritional quality of an area from publicly available well-being data.

Reasons for this could be:

* Lack of training and test data.
* The models we fitted are not appropriate to this task, or more thorough hyperparameter tuning needed.
* Wards have a too high geographic granularity, smoothen out interesting effects.
* There is no underlying effect between well-being factors and nutripoints (nutritional quality of an area), or effect is very small.

## VI) Additional Analyses

### A) Nutripoints Analysis

Now, we will try to better understand how the health score (nutripoints) of the average products are distributed through the areas in London, how certain nutritional variables can influence it and what is the average nutrients composition of the average product of all the areas. 

We have decided to create the **nutripoints** variable to facilitate the prediction. Indeed, it is easier to predict one variable than all the nutritional ones. It therefore represents the healthiness of the average product of a specific area by taking into account the different nutrients it is based on.  

We based our computation of the nutripoints on the definition of the national french Nutri-Score. The bigger it is, the worst is the average product regarding the level of sugar, saturate, sodium and total energy it has. On the other hand, if it is negative (which is almost impossible), it represents products with higher level of proteins and fibers than "bad nutrients" (salt, sugar, saturate). We adapted the different ranges of the official formula to our database as we have average products and therefore very close data ranges. You can see the detailed function in the file: `utils.py`

Moreover, we realised than when summing the main nutrients (carb, protein, fibre, fat, salt), we didn't reach either the average product's weight or 100. Therefore, we rescaled all our values for a 100g in order to be able to compare the different areas. 

#### Data Imports

We import `year_grocery` in order to obtain the weight of each nutrients and to compute the total weight for the typical London product analysis.

We also use `wellbeing_grocery` from the `main` notebook where `nutripoints` and `nutri_class` are already computed. However, we select only the columns referring to nutrition variables.

Usually perceived as "bad" nutrients, salt and sugar are consumed excessively in the total proportion of intakes. On the other hand, saturate, also classified as an unhealthy nutrient is consumed reasonably with a value that doesn't exceed the recommendations. 

In [ ]:
# Data Imports

year_grocery = pd.read_csv("data/year_osward_grocery.csv")
display(year_grocery.head())

wellbeing_grocery = pd.read_pickle("data/wellbeing_grocery.pkl")

grocery_analysis = pd.read_pickle("data/grocery_nutripoints.pkl")
display(grocery_analysis.head())
print(grocery_analysis.shape)

#### Adding labels and adding color

In order to study better the repartition of these Nutritpoints, we will create two columns: one with the label corresponding to each cluster and another one with the corresponding color. 

We based our label and color assessment on the French Nutri-Score scale and rules.

**Let's have a closer look at the nutrients associated with an unhealthy diet: salt, sugar and saturate:**

In [ ]:
grocery_analysis.groupby("nutri_class").min()

#### Nutrilabel

In [ ]:
# Converting nutri class (from 0 to 4) into letters (from A to E) called the nutrilabel
def nutri_labels_to_letter(row_list):
    if row_list == 0:
        return "A"
    elif row_list == 1:
        return "B"
    elif row_list == 2:
        return "C"
    elif row_list == 3:
        return "D"
    elif row_list == 4:
        return "E"
    else:
        return ""


grocery_analysis["nutrilabel"] = grocery_analysis.apply(
    lambda row: nutri_labels_to_letter(row["nutri_class"]), axis=1)

#### Nutricolor

In [ ]:
# Function to set a color to each label
def addcolor(row_list):
    if row_list == "A":
        return "#038141"
    elif row_list == "B":
        return "#85BB2F"
    elif row_list == "C":
        return "#FECC02"
    elif row_list == "D":
        return "#EE8300"
    elif row_list == "E":
        return "#E63F11"
    else:
        return ""


grocery_analysis["color"] = grocery_analysis.apply(
    lambda row: addcolor(row["nutrilabel"]), axis=1)
display(grocery_analysis.head())

#### Exploratory data analysis

We now try to understand how the nutripoints are distributed. We also want to know the density of each nutrilabel.
Moreover, we thought it was important to visualise the correlation between the Nutripoints and each energy as we based our calculations on nutrients weight. The scatter plots is another way to observe the impact of the energy on Nutripoints. 

##### Nutripoints distribution  

In [ ]:
# Explanatory data analysis: distribution of the different variables by counting the number of observations
plt.figure(figsize=(15, 8))

sns.distplot(grocery_analysis["nutripoints"])
plt.xlabel('nutripoints')
plt.ylabel('density')
plt.title('nutripoints distribution')
plt.show();

In [ ]:
# checking if nutripoints follow a normal distribution
print(diagnostic.kstest_normal(grocery_analysis.nutripoints, dist='norm'))

Here, the p_value < 0.05 -> we can reject the null hypothesis that the nutripoints comes from a normal distribution! We could actually observe it on the plot as the curve was not very smooth. 

We can observe that most of the values are between 4.5 and 12. There is not a lot of variance between our different average products. This explains why we are not able to observe precise clusters. Indeed, we force a bit the separation of the areas in different clusters. 

Energy_total is the most correlated to nutripoints with an R value of 0.9. It is followed by carb and sugar. A potent explanation could be that the C points in our Nutripoints computation, corresponding to the "bad" points have more weight than the good ones. Indeed, those values are higher, there is always more fat and carbs than there is protein and fibre. 

Furthermore, we can see that entropy is negatively correlated with Nutripoints. Therefore, an average product with equal proportion of each nutrient leads to good Nutripoints. This is explained by the fact that Nutripoints value fibre and protein and on the contrary when there is too much saturate, sugar or salt, it is badly classified. As a good entropy rhymes with the same level of each nutrient: fiber, protein, fat and carb, the points C will be close to points A, leading to small Nutripoints and a good label.

##### Nutrilabels distribution 

In order to know, which class has the more area, we plot the nutrilabel distribution. Indeed, it could help later to know if well-being variables play a role in that distribution.

In [ ]:
# Plot of the nutrilabel density distribution
plt.figure(figsize=(6, 4))

plt.hist(x=grocery_analysis["nutrilabel"])

plt.xlabel('nutrilabel')
plt.ylabel('density')
plt.title('Nutrilabel Distribution')

plt.show();

The boundary labels are the ones with fewer area. Indeed, most areas are aggregated and spread between B, C and D. C is the one with the most areas. It is actually quite bad because it means that most areas have "unhealthy" average product with mainly salt, sugar and/or fat. If we sum A and B on one hand and C, D and E on the other side, we can observe that the second sum is higher than A+B. Therefore, most of the average products are in the less healthy classes. However, we should not forget our Nutripoints distribution where we could observe that there was no big variance between the Nutripoints. Therefore, the boundaries between each label are not very clear and the difference between labels is actually very small. Assessing that C is not healthy whereas B is actually criticable,  we can only say that B products have mainly a better nutrition value than C products.

##### Merging to have the Index Score

We add to our original database `wellbeing_nutripoints` a column with the Total Wellbeing Score per area.

In [ ]:
wellbeing_nutripoints = pd.merge(left=grocery_analysis, right=wellbeing_grocery[['area_id', 'Index Score 2013']],
                                 on="area_id")
wellbeing_nutripoints.to_pickle("plot_data/plot_wellbeing_nutripoints.pkl")
display(wellbeing_nutripoints.head())

In [ ]:
# Plotting the Wellbeing Total Score in function of the nutripoints
plt.figure(figsize=(14, 3))

plt.scatter(wellbeing_nutripoints["nutripoints"],
            wellbeing_nutripoints["Index Score 2013"], c=wellbeing_nutripoints["color"])

plt.xlabel("Nutripoints")
plt.ylabel("Well-being Index Score 2013")
plt.title("Well-being Total Score in funtion of the nutripoints")
plt.show()

From the above plot, we can better visualize the repartition of the nutrilabels. However, we cannot observe distinct clusters. Indeed, Nutripoint seems to follow a continuous distribution as almost values in the range of 1 to 15 have is linked to a Well-being Score.

This plot confirms that nutrition is not well correlated to well-being. Indeed, well-being index score is not decreasing with the Nutripoints. Some points in B have a very low Well-being Score while a point in E has high Index Score. In each nutrilabel, we have all sort of Index Score. Therefore, we cannot make conclusions on the impact of well-being on nutritional variables.

With the merging, we lost our outliers points from A label because we did not have their well-being value. 

#### Nutrients energy and nutripoints correlation and influence

As we have seen, it is difficult to understand the effect of well-being on nutrition. We however want to observe the correlation between the nutrient energy and the Nutripoints as each nutrient play a role in the computation of this variable.

In [ ]:
# Computing the correlation between the different variables from the dataset according to the spearman method
correlation = grocery_analysis.corr(method="spearman")
display(correlation)

In [ ]:
# Computing and plotting the correlation between the different nutrients and the nutripoints
plt.figure(figsize=(14, 3))
correlation["nutripoints"].plot.bar(x=None, y=None, width=0.8, legend=None)
plt.ylabel("Spearman R")
plt.title("Correlation nutripoints")
plt.show()

Energy_total is the most correlated to nutripoints with an R value of 0.9. It is followed by carb and sugar. A potent explanation could be that the C points in our Nutripoints computation, corresponding to the "bad" points have more weight than the good ones. Indeed, those values are higher, there is always more fat and carbs than there is protein and fibre. 

Furthermore, we can see that entropy is negatively correlated with Nutripoints. Therefore, an average product with equal proportion of each nutrient leads to good Nutripoints. This is explained by the fact that Nutripoints value fibre and protein and on the contrary when there is too much saturate, sugar or salt, it is badly classified. As a good entropy rhymes with the same level of each nutrient: fiber, protein, fat and carb, the points C will be close to points A, leading to small Nutripoints and a good label.

In [ ]:
# Representation of ward areas according to their values for food-related predictor pairs
# Colors are set according to the nutrilabel of the area

fig, ax = plt.subplots(3, 4, figsize=(16, 8), sharey=False)

for i in range(1, 11):
    sbplt = ax[int(i/4), i % 4]
    sbplt.scatter(
        grocery_analysis.iloc[:, i], grocery_analysis["h_nutrients_calories"], c=grocery_analysis["color"])
    sbplt.set_xlabel('')
    sbplt.set_ylabel('')
    sbplt.set_title(grocery_analysis.columns[i], wrap=True)

fig.tight_layout()
fig.subplots_adjust(top=0.9)

fig.suptitle(
    'Scatter plots of the different areas for food-related predictor pairs', fontsize=18);

When representing entropy in function of each nutrient or nutripoints, we can observe their impact on this variable.

For instance, a low energy product would have a higher entropy than one with high energy. Indeed, high energy often means too much fat or carb which are very caloric taking place for other nutrients. It seems that most of A and B have lower total energy and therefore higher entropy. This decreasing trend can be observed for sugar and carb too but it is less distinct for fat and saturate. 

Moreover, increasing trend is more difficult to assess for protein and fibre. They are more aggregated in a corner of the graph.

When observing the Nutripoints figure, we can assess that indeed, the lowest entropy are part of D and E classes. 

##### Typical Londoner product analysis 

We want to analyse the repartition of the different nutrients in an average product. 

We compute therefore the average total_weight and average weight of each nutrients. We then represent them in a pie chart to visualize it better. 

We will try to compare this repartition with the reference intakes recommendation set by the European law. 

In [ ]:
# Main classes of nutrients
NUTRIENTS = ["fibre", "protein", "carb", "fat", "salt"]
SUBNUTRIENTS = ["sugar", "saturate"]

In [ ]:
# Creating a dataframe with all the nutrient's weight from year_grocery
weight = year_grocery[["area_id"]+NUTRIENTS].copy()

In [ ]:
# Computing the total nutrient's weight
weight["weight_total"] = weight[NUTRIENTS].sum(axis=1)
display(weight.head())

In [ ]:
# Adding saturate and sugar values after computing the total weight since they are subcategories of nutrients
weight[['sugar', 'saturate']] = year_grocery[["sugar"] + ["saturate"]]
weight['carb_not_sugar'] = year_grocery["carb"]-year_grocery["sugar"]
weight['fat_not_saturate'] = year_grocery["fat"]-year_grocery["saturate"]
weight['not_saturate_sugar_salt'] = weight["weight_total"] - \
    year_grocery["saturate"]-year_grocery["sugar"]-year_grocery["salt"]
display(weight.head())

In [ ]:
# Computing the features for typical londonner product based on the average weight of each nutrients
weight_mean = weight.mean(axis=0)
display(weight_mean)

In [ ]:
# Pie chart, where the slices will be ordered and plotted counter-clockwise:
nutrients_labels = ["Fiber", "Protein", "Carbohydrates", "Fat", "Salt"]
colors = ["limegreen", "red", "skyblue", "gold", "royalblue"]
fig1, ax1 = plt.subplots(figsize=(10, 8))
ax1.pie(weight_mean[0:5], radius=1, labels=nutrients_labels,
        autopct='%1.1f%%', startangle=90, colors=colors, textprops={'fontsize': 14}, pctdistance=0.85)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title('Proportion of nutrients in the average product',
          fontsize=22, y=1.05, fontweight="bold")
plt.tight_layout(pad=0)
path = '../xavoliva6.github.io/img'
plt.savefig(f"{path}/proportion_nutrients_average_london.svg", format='svg',
            transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

This pie chart visually shows the proportion of each nutrient in the average product in London. It will be compared to the recommendations for a healthy diet given by the [European law](https://www.nutrition.org.uk/attachments/article/234/Nutrition%20Requirements_Revised%20Oct%202016.pdf.)

Recommendations:   
Fats: 14%  
Fibres: 14%  
Carbs: 52%  
Protein: 10%  
Sodium: 1.2%
Sugar: not more than 5%   
Saturate: not more than 11%   

By comparing the recommendations with the average product values, we find out that people in London eat too much fat and proteins. Moreover, the proportion of fibre in the Londoner diet is well below the recommendations. What about sugar, salt and saturate?

**Let's have a closer look at the nutrients associated with an unhealthy diet: salt, sugar and saturate:**

In [ ]:
weight_sugar_saturate_salt = weight_mean[[
    "saturate", "sugar", "salt", "not_saturate_sugar_salt"]]

colors = ["gold", "sienna", "royalblue", "silver"]

subnutrients_labels = ["Saturate fats", "Sugar", "Salt", "Others"]
fig1, ax1 = plt.subplots(figsize=(10, 8))
ax1.pie(weight_sugar_saturate_salt, radius=1, labels=subnutrients_labels,
        autopct='%1.1f%%', startangle=90, colors=colors, textprops={'fontsize': 14}, pctdistance=0.7)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title(
    'Proportion of negative nutrients in the average product', fontsize=22, y=1.05, fontweight="bold")

plt.savefig(f"{path}/proportion_salt_sugar_saturate_average_london.svg", format='svg',
            transparent=True, bbox_inches='tight', pad_inches=0)
plt.show()

Usually perceived as "bad" nutrients, salt and sugar are consumed excessively in the total proportion of intakes. On the other hand, saturate, also classified as an unhealthy nutrient is consumed reasonably with a value that doesn't exceed the recommendations.

### B) House Prices Analysis

#### Data imports

In [ ]:
NUTRITION_COLS = ["area_id", "energy_tot", "energy_fat", "energy_saturate", "energy_sugar", "energy_protein",
                  "energy_carb", "energy_fibre", "energy_alcohol", "h_nutrients_calories"]

year_grocery = pd.read_csv("data/year_lsoa_grocery.csv")
print(year_grocery.shape)
display(year_grocery.head())

In [ ]:
min_grocery = year_grocery[['energy_tot', 'saturate', 'salt',
                            'sugar', 'f_fruit_veg', 'fibre', 'protein']].min(axis=0)

max_grocery = year_grocery[['energy_tot', 'saturate', 'salt',
                            'sugar', 'f_fruit_veg', 'fibre', 'protein']].max(axis=0)

year_grocery["nutripoints"] = year_grocery.apply(
    lambda row: calculate_nutripoints(row, min_grocery, max_grocery), axis=1)

In [ ]:
COLS = ["Code", "Year ending Dec 2014"]

housing_prices = pd.read_excel(
    "data/land-registry-house-prices-LSOA.xls", sheet_name="Mean")[COLS]
housing_prices.rename(
    columns={"Year ending Dec 2014": "mean house price"}, inplace=True)
housing_prices.dropna(inplace=True)

print(housing_prices.shape)
display(housing_prices.head())

#### Merge datasets

In [ ]:
grocery_housing = pd.merge(
    year_grocery, housing_prices, left_on="area_id", right_on="Code")
grocery_housing.drop("Code", axis=1, inplace=True)

In [ ]:
grocery_housing['mean house price'] = pd.to_numeric(
    grocery_housing['mean house price'], errors='coerce')
display(grocery_housing)

#### Exploratory Data Analysis

In [ ]:
ax = sns.histplot(data=grocery_housing, x="mean house price")

print(grocery_housing["mean house price"].median())

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

sns.ecdfplot(data=grocery_housing, x="mean house price",
             complementary=True, ax=ax[0])
sns.ecdfplot(data=grocery_housing, x="mean house price",
             complementary=True, ax=ax[1])
ax[1].set(xscale="log", yscale="log")


price_mean = grocery_housing["mean house price"].mean()
price_median = grocery_housing["mean house price"].median()

print(price_mean, price_median, price_mean / price_median)

The data is skewed to the right, with a long tail of high scores pulling the mean up more than the median. According to the graphs, the data clearly does not follow a normal distribution.

In [ ]:
COLS = ["energy_tot", "energy_fat", "energy_saturate", "energy_sugar", "energy_protein",
        "energy_carb", "energy_fibre", "energy_alcohol", "h_nutrients_calories", "nutripoints"]

correlation = grocery_housing.corr(method="spearman").loc[COLS]

correlation['mean house price'].plot.bar(
    x=None, y=None, width=0.8, legend=None)

plt.title("Mean house price")
plt.ylabel("spearman correlation")

In [ ]:
COLS_CORRELATED = ["energy_tot", "energy_sugar", "energy_carb", "energy_fibre", "h_nutrients_calories", "nutripoints"]
colors = ["r", "r", "r", "g", "g", "r"]

N = len(COLS_CORRELATED)

fig, ax = plt.subplots(2, 3, figsize=(25, 10))

for i, col in enumerate(COLS_CORRELATED):
    ax[int(i / 3), i % 3].scatter(grocery_housing["mean house price"],
                    y=grocery_housing[col], c=[colors[i]] * len(grocery_housing))
    ax[int(i / 3), i % 3].set(xscale="log", title=col, xlabel="mean house price")
    
fig.tight_layout()

Nutripoints are negatively related to mean house prices. The graph on the ri

In [ ]:
house_prices_median = grocery_housing["mean house price"].median()


def classify_median(row, median):
    if row["mean house price"] > median:
        return "high"
    else:
        return "low"


grocery_housing["pricing"] = grocery_housing.apply(
    lambda i: classify_median(i, house_prices_median), axis=1)

In [ ]:
grocery_housing[['nutripoints', 'pricing']
                ].boxplot(by='pricing', figsize=(10, 5))

plt.show()

#### Predictive Models

In [ ]:
scaler = StandardScaler()
grocery_housing.dropna(axis=0, inplace=True)
grocery_housing_stand = grocery_housing.copy()
grocery_housing_stand[["mean house price", "nutripoints"]] = scaler.fit_transform(
    grocery_housing[["mean house price", "nutripoints"]])

X = grocery_housing_stand[["mean house price"]]
y = grocery_housing_stand["nutripoints"]

In [ ]:
# Create the models

lin_reg = LinearRegression()
gb_boost_reg = GradientBoostingRegressor(learning_rate=0.1, n_estimators=100)
ridge_reg = Ridge(alpha=.5)
mlp_reg = MLPRegressor(solver='lbfgs', alpha=1e-5,
                       hidden_layer_sizes=(5, 2))
dt_reg = DecisionTreeRegressor()

mse_scores = {}

reg_models = [lin_reg, gb_boost_reg, ridge_reg, mlp_reg, dt_reg]

mse = 'neg_mean_squared_error'

for model in reg_models:
    model_scores = cross_validate(model, X, y, cv=5, scoring=[mse])

    mse_scores[type(model).__name__] = model_scores["test_" + mse]

pp.pprint(mse_scores)

In [ ]:
mse_scores_df = pd.DataFrame(mse_scores).abs()

In [ ]:
ax = mse_scores_df.plot.bar(figsize=(15, 5))

In [ ]:
best_model = mse_scores_df.mean(axis=0).idxmin()

print(best_model)

### C) Fast-Food Chains Analysis

As we have studied if the nutritional properties of the average product per area were correlated to the well-being features, we now want to study the correlation between the nutrients composition and the number of fast-food restaurants per area.

We were able to find the database from the [London Datastore](https://data.london.gov.uk/), the same one as for the well-being features. 

#### Data Import

In [ ]:
fast_food = pd.read_excel(
    "data/fast_food_ward.xlsx", sheet_name="Ward Data", header=[3], usecols="E,G")
display(fast_food.head())
print(fast_food.shape)

wellbeing_grocery = pd.read_pickle("data/wellbeing_grocery.pkl")
display(wellbeing_grocery.head())
print(wellbeing_grocery.shape)

grocery_analysis = pd.read_pickle("data/grocery_nutripoints.pkl")
display(grocery_analysis.head())
print(grocery_analysis.shape)

#### Data Merging

We compare the number of lines of grocery dataset and fast-food one and try to see how many they have in common. We then merge the two datasets. We also check that there is no null or NA values.

In [ ]:
# check if area id is a unique id
is_area_unique = grocery_analysis["area_id"].is_unique
print(is_area_unique)

In [ ]:
nr_wards_grocery = len(set(grocery_analysis["area_id"].values))
print(nr_wards_grocery)

In [ ]:
grocery_analysis.isna().any()

In [ ]:
nr_wards_fast_food = len(set(fast_food["2015 Ward code"].values))
print(nr_wards_fast_food)

In [ ]:
fast_food.isna().any()

In [ ]:
fast_food.isnull().any()

In [ ]:
# calculate number of rows both datasets have in common

nr_wards = len(set(fast_food["2015 Ward code"].values)
    & set(grocery_analysis["area_id"].values))

print(nr_wards)

We loose 70 values by merging.  

In [ ]:
fastfood_grocery = pd.merge(
    left=grocery_analysis, right=fast_food, left_on='area_id', right_on="2015 Ward code")
fastfood_grocery = fastfood_grocery.drop("2015 Ward code", axis=1)
display(fastfood_grocery.head())
print(fastfood_grocery.shape)

We also merge well-being and fast-food in order to check the correlation between the well-being features and the number of fast-food, as we can make some assumptions about how they are linked and maybe prove the reliability of our data. 

In [ ]:
fastfood_wellbeing = pd.merge(
    left=wellbeing_grocery, right=fast_food, left_on='area_id', right_on="2015 Ward code")
fastfood_wellbeing = fastfood_wellbeing.drop("2015 Ward code", axis=1)
display(fastfood_wellbeing.head())
print(fastfood_wellbeing.shape)

We lose 33 values between well-being and grocery because wellbeing_grocery had already been merged before with year_grocery, leaving aside the area for which we did not have the well-being values.

#### Comprehension of the data

##### Distribution of the values: describe, boxplot, distplot

In [ ]:
# Understanding better how the values are distributed
fastfood_grocery.describe()

In [ ]:
fig = plt.figure(figsize=(4, 5))

sns.boxplot(y=fastfood_grocery["Count of outlets"])

They are many outliers for the number of outlets going up to 147 whereas the median is at 11. Due to this phenomenom, the mean is quite high whereas 75% of the values are between 1 and 18. 

In [ ]:
# List of columns of interest in the nutritional dataset
COLUMNS_GROCERY = [
    'energy_fat',
    'energy_saturate',
    'energy_sugar',
    'energy_protein',
    'energy_carb',
    'energy_fibre',
    'energy_alcohol',
    'energy_tot',
    'h_nutrients_calories',
    'nutripoints',
    'Count of outlets'
]

# Selection of the numerical columns of interest in the fastfood_grocery dataset
fastfood_grocery_analysis = fastfood_grocery[COLUMNS_GROCERY].copy()

In [ ]:
fig, ax = plt.subplots(4, 3, figsize=(16, 8), sharey=False)

for i in range(len(COLUMNS_GROCERY)):
    sbplt = ax[int(i/3), i % 3]

    sns.histplot(data=fastfood_grocery_analysis.iloc[:, i], ax=sbplt)
    sbplt.set_xlabel('')
    sbplt.set_ylabel('')
    sbplt.set_title(fastfood_grocery_analysis.columns[i], wrap=True)

fig.tight_layout()
fig.subplots_adjust(top=0.9)

fig.suptitle('Histplot for each column', fontsize=18)

Most of the nutritional variables seem to be normally distributed. On the other side, the number of outlets is more logarithmic. Most of the areas have between 5 and 10 fast foods but it goes up to 140! 

##### Correlation between the number of fast foods and the different nutritional variables

In [ ]:
# Heatmap to visualize the correlation between the variables
fig = plt.figure(figsize=(10, 6))
sns.heatmap(fastfood_grocery_analysis.corr())

The correlation with the number of oulets (the last column or row) seems really low as the colours are mainly red, corresponding to values around 0. We will display the correlation table for the count of outlets to better understand the importance of the correlation with the nutritional variables. 

In [ ]:
correlation = fastfood_grocery_analysis.corr(method="spearman")
display(correlation[["Count of outlets"]])

In [ ]:
plt.figure(figsize=(14, 3))
correlation["Count of outlets"].plot.bar(
    x=None, y=None, width=0.8, legend=None)
plt.ylabel("Spearman R")
plt.title("Correlation fast food outlets")
plt.show()

**The number of fast food is not correlated at all with the Nutripoints.**   
Furthermore, there is almost no correlation with other variables. Therefore, it is hardly justifiable to predict the nutritional informations of the average product of an area from the number of fast-food.

We will however try to further investigate this correlation. 

##### Correlation between the number of fast food and the well-being measures

In [ ]:
# List of columns of interest in the wellbeing dataset
COLUMNS_SCORES = [
    'Life Expectancy',
    'Childhood Obesity',
    'Incapacity Benefit rate',
    'Unemployment rate',
    'Crime rate - Index',
    'Deliberate Fires',
    'Average Capped GCSE and Equivalent Point Score Per Pupil',
    'Unauthorised Absence in All Schools (%)',
    'Dependent children in out-of-work families',
    'Public Transport Accessibility',
    'Homes with access to open space & nature, and % greenspace',
    'Subjective well-being average score',
    'Index Score 2013',
    'nutripoints',
    'Count of outlets'
]

# Selection of the numerical columns of interest in the wellbeing_grocery dataset
fastfood_wellbeing_analysis = fastfood_wellbeing[COLUMNS_SCORES].copy()

In [ ]:
correlation = fastfood_wellbeing_analysis.corr(method="spearman")
display(correlation[["Count of outlets"]])

For the plot, we only keep the variables that have a spearman score higher than 0.2.

In [ ]:
fastfood_wellbeing_analysis.drop(columns=['Incapacity Benefit rate',
                                          'Deliberate Fires',
                                          'Average Capped GCSE and Equivalent Point Score Per Pupil',
                                          'Unauthorised Absence in All Schools (%)', 'Dependent children in out-of-work families',
                                          'Homes with access to open space & nature, and % greenspace',
                                          'Subjective well-being average score'], inplace=True)
fastfood_wellbeing_analysis.to_pickle("plot_data/fastfood_wellbeing.pkl")

In [ ]:
# Heatmap to visualize the correlation between the variables
fig = plt.figure(figsize=(10, 6))
sns.heatmap(fastfood_wellbeing_analysis.corr())

In [ ]:
correlation_shorten = fastfood_wellbeing_analysis.corr(method="spearman")

In [ ]:
plt.figure(figsize=(14, 3))
correlation_shorten.iloc[0:7, 7].plot.bar(
    x=None, y=None, width=0.8, legend=None)
plt.ylabel("Spearman R")
plt.title("Correlation fast food outlets-wellbeing")
plt.show()

When talking about fast-food and its relation with well-being variables, it is true that we already have some assumptions in our head. Of course, it is well correlated with Public Transport as to facilitate the access to the public and increase customer rate. On the other hand, Crime rate should be low (as we can see here it is negatively correlated to the number of fast-food) as the clients should feel safe when coming to the restaurants. However, we would have expected that it would be positively correlated with Childhood Obesity and Unemployment rate. We think that as fast-food are cheap, the chains won't target the fancy social class but mostly the ones that need comfort foods. Furthermore, fast-food don't serve healthy food but mostly fat and sugar riched one, causing obesity, so we thought that it was positively correlated with this variable. 

Therefore, some results seem predictable but others aren't. Thus, it is difficult to make a decisive conclusion about the link between fast-food and the Nutripoints of the average product of an area. We then do a regression analysis to see if it justifiable to try to predict Nutripoints from the number of fast-food in an area.

#### Regression Analysis

##### Linear Regression of Nutripoints from the number of fast-food

In [ ]:
# Understanding the linear regression between the two variables
Y = fastfood_grocery[["nutripoints"]]
X1 = fastfood_grocery[["Count of outlets"]]
X = sm.add_constant(X1)  # adding a constant

model = sm.OLS(Y, X).fit()
predictions = model.predict(X)

print_model = model.summary()
print(print_model)

The R-squared is very small: 0.005, meaning we can only predict 0.5% of our Nutripoints from the number of fast food. Furthermore, the coefficient of this latter is neglectable as it is -0.0137, the intercept is the one having the more weight in this regression. Therefore, we can conclude that the Linear Regression is not the adapted regression for prediction of Nutripoints of each area from the number of fast-food. It actually makes sense as there was no correlation between these two variables, it is actually difficult to predict one from the other. 

##### Gradient Boosting Regressor

We know that there is no correlation and that the Linear Regression is not representative at all. However, we still wanted to explore if another type of regression: the Gradient Boosting method, would be more concluent. 

In [ ]:
# train a gradient boosting regressor
gradboost = GradientBoostingRegressor()

In [ ]:
y = fastfood_grocery["nutripoints"]
predicted_y = cross_val_predict(gradboost, X, y, cv=5)

In [ ]:
# Plot the results
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(Y, predicted_y, edgecolors=(0, 0, 0))
ax.set_xlabel('Original')
ax.set_ylabel('Predicted')
plt.show()

If the prediction was representative of the nutripoints, we should observe a diagonal line, it is not what we have at all. This means that for a certain Nutripoints value, the predicted one from the Gradient Boosting Regression is higher or smaller than the original value. 

To confirm this assumption, we compute the mean squared error and the R-squared value:

In [ ]:
r2 = r2_score(Y, predicted_y)
mse = mean_squared_error(Y, predicted_y)

print(r2, mse)

The R-squared is negative. A horizontal line would actually be more representative of our regression. Furthermore, the mean squared error is very high. The regression is therefore not representative neither of our Nutripoints variable. 

#### Conclusion 

As the correlation predicted it, it is not possible to find a good regression in order to predict the Nutripoints from the number of fast food of an area. Indeed, there are almost not correlated, making it difficult to have high coefficient for our regressions. 

Having the average products per area, actually, makes it difficult to have a representation of how healthy the inhabitants of an area consume. A high Nutripoint doesn't necessary mean that all the consumers of this area consume more fatty or sugary products. Furthermore, an unhealthy average product is not linked to a high number of fast-foods. However, it could be interesting to see if the richest areas have less fast-foods making the assumption that the inhabitants would prefer to go in a real restaurant.  